In [0]:
from pyspark.sql.functions import col,year,month,dayofmonth,to_date,from_json,current_timestamp
from pyspark.sql.types import *
from pyspark.sql import *
from datetime import datetime
from zoneinfo import ZoneInfo

In [0]:
spark.conf.set(
    "fs.azure.account.key.bmwstorageacc.dfs.core.windows.net",
    dbutils.secrets.get(scope = "bmwanalytics", key = "bmwstorevalut")
)

In [0]:
Event_NameSPACE = "bmwanalytics"
Event_HOSTNAME = "bmwspace.servicebus.windows.net"

CONNECTION_STRING = dbutils.secrets.get(scope = "bmwanalytics", key = "bmwnamevalut")

procuder = {
    "kafka.bootstrap.servers":f"{Event_HOSTNAME}:9093",
    "subscribe": f"{Event_NameSPACE}",
    "kafka.security.protocol":"SASL_SSL",
    "kafka.sasl.mechanism":"PLAIN",
    'kafka.sasl.jaas.config':f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{CONNECTION_STRING}";',
    "startingOffsets": "latest",
    "failOnDataLoss": "false",
}

In [0]:
PIPELINE_VERSION = "V-"+datetime.now(ZoneInfo("Asia/Kolkata")).strftime("%Y-%m-%d %H:%M:%S")
print(PIPELINE_VERSION)

In [0]:
storage = "abfss://bmwstorage@bmwstorageacc.dfs.core.windows.net"
bronze_store = f"{storage}/bronze"
snapshot = f"{storage}/bronze/snapshot/{PIPELINE_VERSION}"

bronze_out = f"{bronze_store}/bmw_sales/sales/{PIPELINE_VERSION}"

print(bronze_out)

In [0]:
ckpt1 = f"{bronze_store}/bmw_sales/sales/{PIPELINE_VERSION}"
ckpt2 = f"{storage}/bronze/snapshot/{PIPELINE_VERSION}"
bronze_ckpt = f"{ckpt1}/_checkpoint"
snapshot_ckpt = f"{ckpt2}/_checkpoint"
print(bronze_ckpt)

In [0]:
raw_df = (
    spark.readStream
    .format("kafka")
    .options(**procuder)
    .load()
)

In [0]:
df_json = raw_df.selectExpr("CAST(VALUE AS STRING) AS raw_json")

In [0]:
query1 = (
    df_json.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", bronze_ckpt)
    .option("overwriteSchema", "true")
    .option("mergeSchema", "true")
    .start(bronze_out)
)

query2 = (
    df_json.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", f"{snapshot_ckpt}")
    .option("overwriteSchema", "true")
    .option("mergeSchema", "true")
    .start(snapshot)
)